Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [ ]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

We add the regularization term $\beta \|w\|^2/2$ to the error function. Let's start with logistic regression.

In [ ]:
beta = 0.001
num_steps = 3001
keep_prob = 1. # Parameter used in a later exercise

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Drop random neurons.
  tf_keep_prob = tf.placeholder(tf.float32)   # FIXME: Returns an error when using tf_keep_prob below
  weights = tf.nn.dropout(weights, keep_prob)
    
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Computing loss.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
    + 0.5 * beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob : keep_prob}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Let's now do the neural network.

In [ ]:
batch_size = 128
num_hidden_units = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Drop random neurons.
  tf_keep_prob = tf.placeholder(tf.float32)  # FIXME: Returns an error when using tf_keep_prob below
  
  # First layer
  biases_hidden_1 = tf.Variable(tf.zeros([num_hidden_units]))
  weights_hidden_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_units]))
  weights_hidden_1_drop = tf.nn.dropout(weights_hidden_1, keep_prob)

  # Last layer
  biases_last = tf.Variable(tf.zeros([num_labels]))
  weights_last = tf.Variable(tf.truncated_normal([num_hidden_units, num_labels]))
  weights_last_drop = tf.nn.dropout(weights_last, keep_prob)

  # Training
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1_drop) + biases_hidden_1)
  logits = tf.matmul(logits, weights_last_drop) + biases_last

  # Compute loss
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
       + 0.5 * beta * tf.nn.l2_loss(weights_hidden_1) \
       + 0.5 * beta * tf.nn.l2_loss(weights_last)
  
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps = 1000, decay_rate = 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
  logits = tf.matmul(logits, weights_last) + biases_last
  valid_prediction = tf.nn.softmax(logits)

  logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
  logits = tf.matmul(logits, weights_last) + biases_last
  test_prediction = tf.nn.softmax(logits)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob : keep_prob}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
beta = 0.001
num_steps = 50
keep_prob = 1 # Parameter used in a later exercise

We see that the error remains very large.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
keep_prob = .5

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
beta = 2*10**(-3)
num_steps = 3001
keep_prob = 1. # Parameter used in a later exercise

In [ ]:
batch_size = 128
num_hidden_units_1 = 1024
num_hidden_units_2 = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Drop random neurons.
  tf_keep_prob = tf.placeholder(tf.float32)  # FIXME: Returns an error when using tf_keep_prob below
  
  # First layer
  biases_hidden_1 = tf.Variable(tf.zeros([num_hidden_units_1]))
  weights_hidden_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_units_1]))
  weights_hidden_1_drop = tf.nn.dropout(weights_hidden_1, keep_prob)
  
  # Second layer
  biases_hidden_2 = tf.Variable(tf.zeros([num_hidden_units_2]))
  weights_hidden_2 = tf.Variable(tf.truncated_normal([num_hidden_units_1, num_hidden_units_2]))
  weights_hidden_2_drop = tf.nn.dropout(weights_hidden_2, keep_prob)

  # Last layer
  biases_last = tf.Variable(tf.zeros([num_labels]))
  weights_last = tf.Variable(tf.truncated_normal([num_hidden_units_2, num_labels]))
  weights_last_drop = tf.nn.dropout(weights_last, keep_prob)

  # Training
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1_drop) + biases_hidden_1)
  logits = tf.nn.relu(tf.matmul(logits, weights_hidden_2_drop) + biases_hidden_2)
  logits = tf.matmul(logits, weights_last_drop) + biases_last

  # Compute loss
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
       + 0.5 * beta * tf.nn.l2_loss(weights_hidden_1) \
       + 0.5 * beta * tf.nn.l2_loss(weights_hidden_2) \
       + 0.5 * beta * tf.nn.l2_loss(weights_last)
  
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps = 1000, decay_rate = 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
  logits = tf.nn.relu(tf.matmul(logits, weights_hidden_2) + biases_hidden_2)
  logits = tf.matmul(logits, weights_last) + biases_last
  valid_prediction = tf.nn.softmax(logits)

  logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
  logits = tf.nn.relu(tf.matmul(logits, weights_hidden_2) + biases_hidden_2)
  logits = tf.matmul(logits, weights_last) + biases_last
  test_prediction = tf.nn.softmax(logits)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob : keep_prob}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

The parameters of the optimizer need to be corrected for this last example.